### Generators and Context Managers

#### Context Manager Pattern

- create context manager
- enter context (and, optionally, receive an object)
- do some work
- exit context  
ie

In [ ]:
with open(file_name) as f:
    data = file.readlines()

#### Mimic Pattern using a Generator

In [ ]:
def open_file(fname, mode):
    f = open(fname, mode)
    try:
        yield f
    finally:
        f.close()

In [ ]:
ctx = open_file('file.txt', 'r')
f = next(ctx) # opens file, and yields it
next(ctx) # closes file -> StopIteration exception

So code might look like this...

In [ ]:
ctx = open_file('file.txt', 'r')
f = next(ctx)

try:
    # do work with file
finally:
    try:
        next(ctx)
    except StopIteration:
        pass

This method works in general...

In [ ]:
def gen(args):
    # do set up work here
    
    try:
        yield object
        
    finally:
        # clean up object here

In [ ]:
ctx = gen(...)
obj = next(ctx)

try:
    # do work with obj
finally:
    try:
        next(ctx)
    except StopIteration:
        pass

But all of this is quite clunky.... you should see that we can almost create a context manager pattern using a generator function!

#### Creating a Context Manager from a Generator Function

In [ ]:
def open_file(fname, mode): # generator function
    f = open(fname, mode) # generator object -> gen = open_file('test.txt', 'w')
    try:
        yield f           # f = next(gen), do work with f
    finally:
        f.close()         # next(f) -> closes f

In [ ]:
class GenContext:
    def __init__(self, gen):
        self.gen = gen
        
    def __enter__(self):
        obj = next(self.gen)
        return obj
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        next(self.gen)
        return False

But we also need a try/except in the exit to catch the StopIteration exception. How this can be done is found in the code below.

So we can use this by doing the following:...

In [ ]:
gen = open_file('test.txt', 'w')
with GenContext(gen) as f:
    # do work

#### Code Examples

In [1]:
def my_gen():
    try:
        print('creating context and yielding object')
        yield [1, 2, 3, 4]
    finally:
        print('exiting context and cleaning up')

In [2]:
gen = my_gen()

In [3]:
lst = next(gen)

creating context and yielding object


In [4]:
lst

[1, 2, 3, 4]

In [5]:
lst

[1, 2, 3, 4]

In [6]:
next(gen)

exiting context and cleaning up


StopIteration: 

In [7]:
gen = my_gen()
lst = next(gen)
print(lst)
try:
    next(gen)
except StopIteration:
    pass

creating context and yielding object
[1, 2, 3, 4]
exiting context and cleaning up


In [8]:
class GenCtxManager:
    def __init__(self, gen_func):
        self._gen = gen_func()
        
    def __enter__(self):
        return next(self._gen)
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        try:
            next(self._gen)
        except StopIteration:
            pass
        return False

In [9]:
def my_gen():
    try:
        print('creating context and yielding object')
        yield [1, 2, 3, 4]
    finally:
        print('exiting context and cleaning up')

In [10]:
with GenCtxManager(my_gen) as obj:
    print(obj)

creating context and yielding object
[1, 2, 3, 4]
exiting context and cleaning up


In [11]:
class GenCtxManager:
    def __init__(self, gen_func, *args, **kwargs):
        self._gen = gen_func(*args, **kwargs)
        
    def __enter__(self):
        return next(self._gen)
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        try:
            next(self._gen)
        except StopIteration:
            pass
        return False

In [14]:
def open_file(fname, mode):
    f = open(fname, mode)
    try:
        print('opening file...')
        yield f
    finally:
        print('closing file...')
        f.close()

In [15]:
with GenCtxManager(open_file, 'test.txt', 'w') as f:
    f.writelines('testing...')

opening file...
closing file...


In [16]:
with GenCtxManager(open_file, 'test.txt', 'r') as f:
    print(f.readlines())

opening file...
['testing...']
closing file...
